In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

Python-dotenv reads key-value pairs from a .env file and can set them as environment variables

In [3]:
from dotenv import load_dotenv

load_dotenv() # take environment variables from .env

True

In [4]:
#1:global approach by setting the key in environment variables
#2:export the key in terminal
#3:create an .env file and paste it there
openai_key=os.getenv("OPENAI_API_KEY3")

#### Create LLM

In [5]:
local_llm=ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo", temperature=0.5)

c:\Users\kotav\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [31]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

#### Create Prompt 1

In [8]:
RESPONSE_JSON = {
    "1": {
        "MCQ": "Multiple Choice Question",
        "Options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "MCQ": "Multiple Choice Question",
        "Options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "MCQ": "Multiple Choice Question",
        "Options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [9]:
TEMPLATE="""
Text: {text}
You're an expert Bible Quiz Maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} \
student in {tone} tone. Make sure the questions are not repeated and check all the questions to be conforming the text as well. \
Make sure to format your response like RESPONSE_JSON below and use it as a guide. \
### RESPONSE_JSON
{response_json}

"""

In [10]:
prompt_template= PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

#### Connect LLM with Prompt 1 using chain

In [11]:
quiz_chain=LLMChain(llm=local_llm, prompt=prompt_template, output_key="quiz", verbose=True)

#### Create Prompt 2

In [12]:
TEMPLATE2="""
You're an expert English Grammarian and writer. Given a Multiple Choice Quiz for {subject} students. \
You need to evaluate the complexity of the question and given complete analysis of the quix. Only use maximum of 50 words. \
If the quiz is not as per the cognitive and analytical abilities of the students, \
update the quiz questions which needs to be changed and change the tone such that it fits perfectly the abilities of the students.
Quiz_MCQs:
{quiz}

Check the above quiz as an expert English Writer
"""

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject","quiz"], template=TEMPLATE2)

#### Connect LLM with Prompt 2 using a chain

In [14]:
review_chain=LLMChain(llm=local_llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

#### Connect both chains

In [15]:
generate_evaluate_cahin=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text","number","subject","tone","response_json"],
                                        output_variables=["quiz","review"], verbose=True)

In [16]:
file_path=r"C:\Users\kotav\mcqgen\data.txt"

In [17]:
file_path

'C:\\Users\\kotav\\mcqgen\\data.txt'

In [18]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [19]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

Machine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised

In [20]:
json.dumps(RESPONSE_JSON)

'{"1": {"MCQ": "Multiple Choice Question", "Options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"MCQ": "Multiple Choice Question", "Options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"MCQ": "Multiple Choice Question", "Options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [21]:
NUMBER=5
SUBJECT="Machine Learning"
TONE="easy"

In [22]:
with get_openai_callback() as cb:
    response=generate_evaluate_cahin(
    {
        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "response_json": json.dumps(RESPONSE_JSON)
    }
    )

c:\Users\kotav\mcqgen\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

Machine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mi

In [23]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1248
Prompt Tokens:865
Completion Tokens:383
Total Cost:0.0020635000000000002


In [24]:
response

{'text': "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\n\nMachine learning approaches have been applied to many fields including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] ML is known in its application across business problems under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) throug

In [25]:
quiz=response.get("quiz")

In [26]:
quiz=json.loads(quiz)

In [27]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["MCQ"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["Options"].items()
        ]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [28]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: The study of artificial intelligence concerned with the development and study of statistical algorithms | b: The study of robotics | c: The study of quantum computing | d: The study of astronomy',
  'Correct': 'a'},
 {'MCQ': 'Which field has machine learning been applied to?',
  'Choices': 'a: Natural language processing | b: Astronomy | c: Chemistry | d: Geology',
  'Correct': 'a'},
 {'MCQ': 'What is computational statistics an important source of in machine learning?',
  'Choices': 'a: Data mining | b: Mathematical optimization methods | c: Artificial neural networks | d: Predictive analytics',
  'Correct': 'b'},
 {'MCQ': 'What does PAC learning provide a framework for describing?',
  'Choices': 'a: Data mining | b: Machine learning | c: Artificial intelligence | d: Quantum computing',
  'Correct': 'b'},
 {'MCQ': 'What is the related field of study focusing on exploratory data analysis through unsupervised learning?',
  'Choices'

In [29]:
Trivia=pd.DataFrame(quiz_table_data)

In [30]:
Trivia.to_csv("machine_learning.csv", index=False)

EOF